### 라이브러리

In [4]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from dotenv import load_dotenv

### API 입력

In [ ]:
load_dotenv()

# API 키 설정
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

### 문서 로드/분할 및 벡터 임베딩

In [ ]:
# PDF 파일 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()

#PDF 파일을 300자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                               chunk_overlap = 20,
                                               )
docs = text_splitter.split_documents(pages)

#텍스트 임베딩 (OpenAI 사용)
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, 
                                   embedding=embeddings,
                                   )
retriever = vectorstore.as_retriever()

### 모델 선언

In [ ]:
# Claude LLM 선언
llm = ChatAnthropic(model = "claude-3-5-haiku-20241022",
                    temperature = 0,
                    max_tokens = 1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트 불러오기
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문 및 답변
question = "평형기능의 장해가 있는거 같아 혹시 기준표나 점수가 있을까?"
answer = rag_chain.invoke(question)
print(answer)


C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


평형기능 장해는 전정기관 이상으로 보행 등 일상생활이 어려운 상태에서 평형장해 평가항목별 합산점수가 30점 이상인 경우를 말합니다. 평가 항목에는 양측 전정기능 소실 14점, 양측 전정기능 감소 10점 등이 포함됩니다. 정확한 장해 판정을 위해서는 의료 전문가의 정밀한 검사와 평가가 필요합니다.
